In [1]:
 %cd /content/drive/MyDrive/colab/newdata

/content/drive/MyDrive/colab/newdata


In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('Spirit.log_templates.csv')
df.head()

,ID,EventId,EventTemplate
0,0,9a640663,<*> (root) CMD (/home/#1#/bin/grab_myrinet_inf...
1,1,fa9c45af,<*> (root) CMD (test -x <*> && <*> cron)
2,2,03badb70,<*> LAuS error - <*> - <*> (19) <*> No such de...
3,3,669e1ffc,dhcpd: DHCPDISCOVER from <*> via <*> network <...
4,4,7b933a56,kernel: hda: drive not ready for command


In [4]:
df = df.values
spirit = {}
EventID = df[:,1]
for i in range(len(df)):
  tmp = str(EventID[i])
  spirit[tmp] = i


In [22]:
template = df[:,2]

In [9]:
log = pd.read_csv('Spirit.log_structured.csv')
log.head()

,LineId,Label,Timestamp,Date,User,Month,Day,Time,Location,Content,EventId,EventTemplate,ParameterList
0,1,-,1104566400,2005.01.01,sadmin1,Jan,1,00:00:00,sadmin1/sadmin1,CROND[30483]: (root) CMD (/home/#1#/bin/grab_m...,9a640663,<*> (root) CMD (/home/#1#/bin/grab_myrinet_inf...,['CROND[30483]:']
1,2,-,1104566400,2005.01.01,sadmin2,Jan,1,00:00:00,src@sadmin2,CROND[32483]: (root) CMD (test -x /etc/pbs_sta...,fa9c45af,<*> (root) CMD (test -x <*> && <*> cron),"['CROND[32483]:', '/etc/pbs_stat.py', '/etc/pb..."
2,3,-,1104566400,2005.01.01,sadmin2,Jan,1,00:00:00,src@sadmin2,CROND[32483]: LAuS error - do_command.c:175 - ...,03badb70,<*> LAuS error - <*> - <*> (19) <*> No such de...,"['CROND[32483]:', 'do_command.c:175', 'laus_lo..."
3,4,-,1104566400,2005.01.01,sadmin2,Jan,1,00:00:00,src@sadmin2,CROND[32483]: LAuS error - do_command.c:226 - ...,03badb70,<*> LAuS error - <*> - <*> (19) <*> No such de...,"['CROND[32483]:', 'do_command.c:226', 'laus_at..."
4,5,-,1104566403,2005.01.01,sadmin1,Jan,1,00:00:03,sadmin1/sadmin1,dhcpd: DHCPDISCOVER from 00:11:85:6b:10:a6 via...,669e1ffc,dhcpd: DHCPDISCOVER from <*> via <*> network <...,"['00:11:85:6b:10:a6', 'eth1:', '172.31.0/16:']"


In [10]:
log = log.values
label = log[:,1]
event = log[:,10]

In [ ]:
true_label = []
true_event = []
cnt = 0
for i in range(len(label)):
  if label[i][0] != '-':
    true_label.append(1)
  else:
    true_label.append(0)
  if event[i] in spirit:
    true_event.append(spirit[event[i]])
  else:
    print(i)
    cnt += 1
    template = np.append(template,log[i,11])
    EventID = np.append(EventID,event[i])
    spirit[event[i]] = len(spirit)
    true_event.append(spirit[event[i]])

In [31]:
###in ra template
#minh co cot template, eventID

print(len(template))
ID = []
for i in range(0, len(template)):
  ID.append(i)
  

2914


In [32]:
spirit_template = pd.DataFrame(columns=['ID','template'])
spirit_template['template'] = template
spirit_template['ID'] = ID

spirit_template.to_csv('spirit_template.csv', index=False, header=True)

In [33]:
sequence_length = 50
sequence = []
final_label = []
tmp = ''
flag = False
for i in range(len(true_label)):
  tmp += str(int(true_event[i]))+' '
  if true_label[i] == 1:
    flag = True
  if (i+1)%sequence_length == 0:
    sequence.append(tmp)
    if (flag == True):
      final_label.append(1)
    else:
      final_label.append(0)
    tmp = ''
    flag = False

In [37]:
spirit = pd.DataFrame(columns=['sequence','label'])
spirit['sequence'] = sequence
spirit['label'] = final_label
spirit.to_csv('spirit_fulllog.csv', index=False, header=True)

In [39]:
from collections import Counter
print(Counter(final_label))
print(Counter(label))

Counter({0: 64611, 1: 35389})
Counter({'-': 4235109, 'R_HDA_NR': 380271, 'R_HDA_STAT': 380271, 'N_PBS_PRE': 1263, 'N_PBS_CHK': 1240, 'N_PBS_EPI': 1164, 'R_EXT_CCISS': 267, 'N_PBS_CON1': 183, 'N_NFS': 62, 'N_GM_MAP': 52, 'N_PBS_BFD1': 51, 'N_PBS_BAIL': 28, 'R_GM_LANAI': 19, 'N_AUTH': 5, 'R_NMI': 5, 'R_GM_PAR4': 3, 'R_GM_PAR2': 2, 'N_OOM': 2, 'R_GM_PAR1': 1, 'R_GM_PAR3': 1, 'R_NMI1': 1})


normal: 4 235 109
abnormal: 764 891

##label sequence
normal: 64 611
abnormal: 35 389 

In [40]:
import tensorflow as tf
from tensorflow import keras

import os
import tempfile

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [41]:
file = tf.keras.utils
raw_df = pd.read_csv('spirit_fulllog.csv')
raw_df.head()

,sequence,label
0,0 1 2 2 3 4 4 5 5 4 4 5 5 6 7 8 9 10 4 4 5 5 3...,1
1,11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 1...,0
2,11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 1...,0
3,11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 1...,0
4,11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 1...,0


In [42]:
raw_df[['sequence', 'label']].describe()

,label
count,100000.000000
mean,0.353890
std,0.478178
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [43]:
neg, pos = np.bincount(raw_df['label'])
total = neg + pos
print('Examples:\n    Total: {}\n    Positive: {} ({:.2f}% of total)\n'.format(
    total, pos, 100 * pos / total))

Examples:
    Total: 100000
    Positive: 35389 (35.39% of total)



In [45]:
cleaned_df = raw_df.copy()
train_df, test_df = train_test_split(cleaned_df, test_size=0.8)
pd.DataFrame(columns=['sequence','label'])
train_df.to_csv('../test/datasets/spirit/train.csv', index = False, header = False, mode = 'a')
test_df.to_csv('../test/datasets/spirit/test.csv', index = False, header = False, mode = 'a')

In [46]:
print(Counter(train_df))
print(Counter(test_df))

Counter({'sequence': 1, 'label': 1})
Counter({'sequence': 1, 'label': 1})
